In [ ]:
# step1 读取数据

In [1]:
# -*- coding:utf-8 -*-
# First, we'll import pandas, a data processing and CSV file I/O library
# import training data
import pandas as pd
t3 = pd.read_csv('E:\Competetion\KDD-cup-2017\KDD_OFFICIAL_DATA\dataSets\\training\links (table 3).csv')
t4 = pd.read_csv('E:\Competetion\KDD-cup-2017\KDD_OFFICIAL_DATA\dataSets\\training\\routes (table 4).csv')
t5_train = pd.read_csv('E:\Competetion\KDD-cup-2017\KDD_OFFICIAL_DATA\dataSets\\training\\trajectories(table 5)_training.csv')
t6_train = pd.read_csv("E:\Competetion\KDD-cup-2017\KDD_OFFICIAL_DATA\dataSets\\training\\volume(table 6)_training.csv")

In [2]:
# import testing data
t6_test_2h_ago = pd.read_csv("E:\Competetion\KDD-cup-2017\KDD_OFFICIAL_DATA\dataSets\\testing_phase1\\volume(table 6)_test1.csv")
t5_test_2h_ago = pd.read_csv("E:\Competetion\KDD-cup-2017\KDD_OFFICIAL_DATA\dataSets\\testing_phase1\\trajectories(table 5)_test1.csv")

In [4]:
t5_test_2h_ago.head()

,intersection_id,tollgate_id,vehicle_id,starting_time,travel_seq,travel_time
0,A,2,1026631,2016-10-18 06:00:14,110#2016-10-18 06:00:14#7.65;123#2016-10-18 06...,27.54
1,C,3,1017312,2016-10-18 06:00:28,115#2016-10-18 06:00:28#9.35;102#2016-10-18 06...,139.53
2,A,2,1044671,2016-10-18 06:03:07,110#2016-10-18 06:03:07#7.22;123#2016-10-18 06...,26.01
3,A,3,1000017,2016-10-18 06:04:37,110#2016-10-18 06:04:37#7.39;123#2016-10-18 06...,78.40
4,A,2,1002850,2016-10-18 06:10:17,110#2016-10-18 06:10:17#14.33;123#2016-10-18 0...,71.00


# step2 按照20分钟的时间窗口重组数据

In [5]:
# 按照时间窗口重组数据: http://stackoverflow.com/questions/36914892/python-how-to-group-pandas-data-frame-in-a-certain-time-window
# step1 将starting time 这一列转换成datetime数据类型
t5_train['starting_time'] =  pd.to_datetime(t5_train['starting_time'] , format='%Y-%m-%d %H:%M:%S')
t5_train.head()

,intersection_id,tollgate_id,vehicle_id,starting_time,travel_seq,travel_time
0,B,3,1065642,2016-07-19 00:14:24,105#2016-07-19 00:14:24#9.56;100#2016-07-19 00...,70.85
1,B,3,1047198,2016-07-19 00:35:56,105#2016-07-19 00:35:56#11.58;100#2016-07-19 0...,148.79
2,B,1,1086390,2016-07-19 00:37:15,105#2016-07-19 00:37:15#5.26;100#2016-07-19 00...,79.76
3,A,2,1071181,2016-07-19 00:37:59,110#2016-07-19 00:37:59#13.74;123#2016-07-19 0...,58.05
4,B,1,1065807,2016-07-19 00:56:21,105#2016-07-19 00:56:21#16.08;100#2016-07-19 0...,137.98


In [6]:
# step2 将时间这一列变成index，为接下来的TimeGrouper('20Min')做准备
t5_train = t5_train.set_index(['starting_time'])
t5_train.head()

,intersection_id,tollgate_id,vehicle_id,travel_seq,travel_time
starting_time,,,,,
2016-07-19 00:14:24,B,3,1065642,105#2016-07-19 00:14:24#9.56;100#2016-07-19 00...,70.85
2016-07-19 00:35:56,B,3,1047198,105#2016-07-19 00:35:56#11.58;100#2016-07-19 0...,148.79
2016-07-19 00:37:15,B,1,1086390,105#2016-07-19 00:37:15#5.26;100#2016-07-19 00...,79.76
2016-07-19 00:37:59,A,2,1071181,110#2016-07-19 00:37:59#13.74;123#2016-07-19 0...,58.05
2016-07-19 00:56:21,B,1,1065807,105#2016-07-19 00:56:21#16.08;100#2016-07-19 0...,137.98


In [7]:
# step3 按照时间、intersection_id、tollgate_id进行聚合求平均值，记住要reset_index()
t5_train = t5_train.groupby([pd.TimeGrouper('20Min'), 'intersection_id', 'tollgate_id']).travel_time.mean()\
       .reset_index().rename(columns = {'travel_time':'avg_travel_time'})
t5_train.head()

,starting_time,intersection_id,tollgate_id,avg_travel_time
0,2016-07-19 00:00:00,B,3,70.85
1,2016-07-19 00:20:00,A,2,58.05
2,2016-07-19 00:20:00,B,1,79.76
3,2016-07-19 00:20:00,B,3,148.79
4,2016-07-19 00:40:00,B,1,137.98


In [8]:
# 对测试数据集进行处理
t5_test_2h_ago['starting_time'] =  pd.to_datetime(t5_test_2h_ago['starting_time'] , format='%Y-%m-%d %H:%M:%S')
t5_test_2h_ago = t5_test_2h_ago.set_index(['starting_time'])
t5_test_2h_ago = t5_test_2h_ago.groupby([pd.TimeGrouper('20Min'), 'intersection_id', 'tollgate_id']).travel_time.mean()\
       .reset_index().rename(columns = {'travel_time':'avg_travel_time'})

In [9]:
t6_train.head()

,time,tollgate_id,direction,vehicle_model,has_etc,vehicle_type
0,2016-09-19 23:09:25,2,0,1,0,NaN
1,2016-09-19 23:11:53,2,0,1,0,NaN
2,2016-09-19 23:13:54,2,0,1,0,NaN
3,2016-09-19 23:17:48,1,0,1,1,NaN
4,2016-09-19 23:16:07,2,0,1,0,NaN


In [10]:
t6_train['time'] =  pd.to_datetime(t6_train['time'] , format='%Y-%m-%d %H:%M:%S')
t6_train = t6_train.set_index(['time'])

# 车流量
t6_train = t6_train.groupby([pd.TimeGrouper('20Min'), 'tollgate_id', 'direction']).size()\
       .reset_index().rename(columns = {0:'volume'})

In [11]:
t6_train.head()

,time,tollgate_id,direction,volume
0,2016-09-19,1,0,13
1,2016-09-19,1,1,140
2,2016-09-19,2,0,2
3,2016-09-19,3,0,17
4,2016-09-19,3,1,181


In [12]:
# 对测试数据集进行处理
t6_test_2h_ago['time'] =  pd.to_datetime(t6_test_2h_ago['time'] , format='%Y-%m-%d %H:%M:%S')
t6_test_2h_ago = t6_test_2h_ago.set_index(['time'])

# 车流量
t6_test_2h_ago = t6_test_2h_ago.groupby([pd.TimeGrouper('20Min'), 'tollgate_id', 'direction']).size()\
       .reset_index().rename(columns = {0:'volume'})